Here we'll show how to compute Delaunay triangulations in 2D.
Under the hood, computing the Delaunay triangulation of a 2D point set is equivalent to computing the 3D convex hull of those points lifted onto a paraboloid in 3-space.
This means that if you understand how convex hulls work, you basically understand how Delaunay triangulations work -- all the moving parts are the same, down to the visibility graph.
First, we'll generate some random input data.

In [ ]:
import numpy as np
from scipy.stats.qmc import PoissonDisk
rng = np.random.default_rng(seed=1729)
num_points = 40
poisson_disk = PoissonDisk(2, radius=0.05, seed=rng)
X = poisson_disk.random(num_points)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.scatter(*X.T);

The plot below shows what these points look like when lifted to a 3D paraboloid.

In [ ]:
from mpl_toolkits import mplot3d
W = np.sum(X**2, axis=1)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.scatter(*np.column_stack((X, W)).T);

Much like for convex hulls, we'll use a state machine object that we'll call `delaunay_machine` to keep track of the progress of the algorithm.

In [ ]:
import zmsh
delaunay_machine = zmsh.delaunay.Delaunay(X)

In [ ]:
from copy import deepcopy
topologies = [deepcopy(delaunay_machine.topology)]

while not delaunay_machine.is_done():
    delaunay_machine.step()
    topologies.append(deepcopy(delaunay_machine.topology))

There is only one extra step for Delaunay triangulations.
If we repurpose an existing algorithm to compute the convex hull of the points lifted up to a parabola, we're going to get two "sides" -- a top and a bottom.
We're only interested in the facets on the bottom of the parabola, so to get the desired output we need to filter out anything on top.
The code below does the filtering for us.

In [ ]:
import predicates

def filter_bottom_facets(topology):
    dimension = topology.shape[1] - 1
    cell_ids_to_remove = []
    for cell_id, cell in enumerate(topology):
        if cell.compressed().size != 0:
            x = X[cell]
            if predicates.volume(x.T) <= 0:
                cell_ids_to_remove.append(cell_id)

    return np.array(
        [
            row
            for index, row in enumerate(topology)
            if row.compressed().size != 0 and not (index in cell_ids_to_remove)
        ]
    )

In [ ]:
ftopologies = [filter_bottom_facets(topo) for topo in topologies]

Now we can see the progress of the algorithm at each step.
Some of the steps are adding facets to the top of the hull of the paraboloid; we'll see those in the animation below as steps that don't appear to make any progress.

In [ ]:
%%capture

from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.axis("off")

def animate(topology):
    ax.clear()
    ax.scatter(*X.T)
    ax.triplot(*X.T, topology)

animation = FuncAnimation(fig, animate, ftopologies, interval=1e3)

In [ ]:
from IPython.display import HTML
HTML(animation.to_jshtml())